In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets
from math import pi
from scipy.optimize import root

# This function calculates a 2D matrix, which transfroms given acceleration vector
def Dipolar_matrix(k, e_dd, phi1):
    if k==1:
        F1=0.8
        F2=0.8
    else:
        y1=np.power(1-k**2, 0.5, dtype=complex)
        F0=np.real(1/y1*np.log((1+y1)/(1-y1)))
        F1=1/4*(10/(1-k**2)-6/((1-k**2)**2)+3*(k**4)/((1-k**2)**2)*F0)
        F2=-1/(1-k**2)*(4-6/(1-k**2)+3/(1-k**2)*(k**2)*F0)
    A1=1-e_dd+3*e_dd*((1-F1)*np.cos(phi1)**2+(3/4*F1)*np.sin(phi1)**2)
    A2=3*e_dd*(1-F1)*np.sin(2*phi1)
    B1=1-e_dd+3*e_dd*((1-F2/2)*np.cos(phi1)**2+(1/4*F2)*np.sin(phi1)**2)
    B2=3/4*e_dd*F2*np.sin(2*phi1)
    return A1, A2, B1, B2

#For a fixed direction of acceleration, this function finds angles along which the magnetic field provides only scaling to an effective vector
def Dipolar_angle(k, gamma1):
    if k==1:
        F1=0.8
        F2=0.8
    else:
        #y1 = ((1-k**2)**(0.5))
        y1=np.power(1-k**2, 0.5, dtype=complex)
        F0=np.real(1/y1*np.log((1+y1)/(1-y1)))
        F1=1/4*(10/(1-k**2)-6/((1-k**2)**2)+3*(k**4)/((1-k**2)**2)*F0)
        F2=-1/(1-k**2)*(4-6/(1-k**2)+3/(1-k**2)*(k**2)*F0)
    A1=4*(F2*np.cos(gamma1)**2-4*(1-F1)*np.sin(gamma1)**2)
    A2=np.sin(2*gamma1)*(3*F2-7*F1)
    A=(A1**2+A2**2)**0.5
    phi_stiff=1/2*np.arcsin((F2-F1)*np.sin(2*gamma1)/A)
    phi_rot=1/2*(np.arctan(A2/A1)+pi*(-np.heaviside(np.sign(np.sin(2*gamma1))*A2/A1,1)))
    phi1=-phi_stiff-phi_rot
    phi2=pi/2+phi_stiff-phi_rot
    return phi1, phi2


#This function builds a general 2D plot in XY plane of how effective acceleration is affected by dipolar interaction    
def plot_ellipse_XZ(k=1.0, e_dd=0.2, phi_deg=0.0, theta_deg=0.0):
  
    phi = np.radians(phi_deg)  
    theta = np.radians(theta_deg)
    f = plt.figure(figsize=(8, 4))
    gs = f.add_gridspec(1, 2, width_ratios=[1, 1])
    ax1 = f.add_subplot(gs[0])
    ax2 = f.add_subplot(gs[1])
    x = np.linspace(-1, 1, 400)
    t0=np.linspace(0,2*pi,200)
    Sc_f=k**(-2/3)
    k_lin=Sc_f*(np.sin(theta)**2+k**(-2)*np.cos(theta)**2)**(-0.5)
    z_cont,x_cont=0*t0, 0*t0
    
    ax1.plot(Sc_f*np.cos(t0), Sc_f*np.sin(t0)*k, 'r')
    ax1.plot(-np.sin(theta)*x*1*k_lin, np.cos(theta)*x*1*k_lin, 'b')
    
    #Calculate and plot lines of zero deflection
    phi1,phi2=Dipolar_angle(k,theta)
    ax1.plot(3*x*np.cos(phi1), 3*x*np.sin(phi1), 'c--')
    ax1.plot(3*x*np.cos(phi2), 3*x*np.sin(phi2), 'c--')
    ax2.plot(2*x*np.cos(phi1), 2*x*np.sin(phi1), 'c--')
    ax2.plot(2*x*np.cos(phi2), 2*x*np.sin(phi2), 'c--')


    #Meshgrid of points inside an ellipse
    zs = np.linspace(-2, 2, 12)
    xs = np.linspace(-2, 2, 12)
    Z, X = np.meshgrid(zs, xs)
    Z_flat = Z.flatten()
    X_flat = X.flatten()
    mask = Z_flat**2 + (X_flat/k)**2 <= Sc_f**2
    points = np.vstack((Z_flat[mask], X_flat[mask])).T
    u = np.cos(phi) * np.ones(len(points))
    v = np.sin(phi) * np.ones(len(points))
    u = u * np.sign(np.cos(theta)*points[:,0]+np.sin(theta)*points[:,1])
    v = v * np.sign(np.cos(theta)*points[:,0]+np.sin(theta)*points[:,1])
    ax1.scatter(points[:,0], points[:,1], color='green', zorder=5, label='Uniform lattice points')
    ax1.quiver(points[:,0], points[:,1], u, v, angles='xy', scale_units='xy', scale=4, color='orange')

    
    #Calculate and plot the curve of the constant acceleration and variated magnetic field
    for i in range(len(t0)):
        A1,A2,B1,B2=Dipolar_matrix(k, e_dd, t0[i])
        Sc_f2=1/(A1*B1-A2*B2)
        z_cont[i] = Sc_f2*(np.cos(theta)*A1-np.sin(theta)*A2)
        x_cont[i] = Sc_f2*(np.sin(theta)*B1-np.cos(theta)*B2)
    ax2.plot(0.5*np.cos(t0), 0.5*np.sin(t0), 'r--')
    ax2.plot(0.5*z_cont, 0.5*x_cont, 'g:')
    
    B_vec=np.array([np.cos(phi),np.sin(phi)])
    A_vec=-np.array([np.cos(theta),np.sin(theta)])
    A1,A2,B1,B2=Dipolar_matrix(k, e_dd, phi)
    Sc_f2=1/(A1*B1-A2*B2)
    A_eff_vec=np.array([Sc_f2*(np.cos(theta)*A1-np.sin(theta)*A2),Sc_f2*(np.sin(theta)*B1-np.cos(theta)*B2)])
    
    # Vectots on the second subplot
    ax2.quiver(0, 0, B_vec[0], B_vec[1], angles='xy', scale_units='xy', scale=2, color='purple')
    ax2.text(0, -0.15, 'B', color='purple', fontsize=10)
    ax2.quiver(0, 0, A_vec[0], A_vec[1], angles='xy', scale_units='xy', scale=2, color='red')
    ax2.text(-0.1, 0.1, 'a', color='red', fontsize=10)
    ax2.quiver(0, 0, A_eff_vec[0], A_eff_vec[1], angles='xy', scale_units='xy', scale=2, color='green')
    ax2.text(0.05, -0.1, r"$\nabla n$", color='green', fontsize=10)


    ax1.set_title('Dipole distribution')
    ax2.set_title('Density gradient')
    ax1.set_xlabel('z')
    #ax1.set_ylabel('x')
    ax1.set_ylabel(r"$\rho$")
    #plt.axis('equal')
    ax1.grid(True)
    ax1.set_xlim(-2,2)
    ax1.set_ylim(-2,2)
    #.axis('equal')
    ax2.set_xlim(-1,1)
    ax2.set_ylim(-1,1)
    plt.show()

# Interactive sliders
# interact(
#     plot_ellipse_XZ,
#     k=(5, 20, 0.01),
#     e_dd=(0, 1, 0.05),
#     phi_deg=(0, 360, 2),
#     theta_deg=(0, 90, 2)
# );
    

interactive(children=(FloatSlider(value=5.0, description='k', max=20.0, min=5.0, step=0.01), FloatSlider(value…